In [94]:
import pandas as pd 
import openpyxl
import yaml
from datetime import datetime, timedelta, date
import pulp
import xlsxwriter
import numpy as np
from ortools.sat.python import cp_model
import calendar
from typing import List, Tuple
import os
from typing import Dict, Any
from dateutil.relativedelta import relativedelta
import re

In [95]:
# proyecto="cabanaconde"
# proyecto="cabanaconde_inicial"
proyecto="cabanaconde_vigente"
# proyecto="callacalla_inicial"
# proyecto="callacalla_vigente"
# proyecto="canocota_inicial"
# proyecto="canocota_vigente"
# proyecto="ccanaseta_inicial"
# proyecto="ccanaseta_vigente"
# proyecto="misanayoc_inicial"
# proyecto="misanayoc_vigente"
# proyecto="sibayo_inicial"
# proyecto="sibayo_vigente"


In [96]:
with open("./data/"+proyecto + '/datos.yaml', 'r',encoding='utf-8') as file:
    datos = yaml.safe_load(file)

print(datos)

{'ruta': 'Emp. R0405105(CABANACONDE)-CHOCO(LD.PROV. CASTILLA)', 'sector': 'Emp. R0405105(CABANACONDE)-CHOCO(LD.PROV. CASTILLA)', 'tramo': 'Emp. R0405105(CABANACONDE)-CHOCO(LD.PROV. CASTILLA)', 'longitud': 13000.0, 'meta': 13000.0, 'fecha_inicio': {'day': 10, 'month': 4, 'year': 2025}, 'tiempo_ejecucion_dias': 240, 'progresiva_inicial': 0.0, 'progresiva_final': 13000.0, 'contratista': 'Grupo ARICOL E.I.R.L', 'categoria': 'Vecinal', 'jefe_mantenimiento': 'Ing. Jorge Luis Huaman Ccahuana', 'numero_trabajadores': 4, 'cuadrilla': 'Única', 'monto': 64896.0, 'numero_cuadrillas': 1, 'servicio': 'Mantenimiento rutinario', 'contrato': '001-2025', 'departamento': 'Arequipa', 'provincia': 'Caylloma', 'distritos': ['Cabanaconde']}


In [97]:
with open("./data/" + proyecto + "/carga_trabajo_reajustada.yaml", "r", encoding="utf-8") as file:
    carga_trabajo = yaml.safe_load(file)

print(carga_trabajo)

{'-actividades': [{'codigo': 'MR-100', 'descripcion': 'CONSERVACION DE CALZADA', 'subactividades': [{'codigo': 'MR-101', 'descripcion': 'Limpieza de Calzada', 'carga_reajustada': 17.33}, {'codigo': 'MR-102', 'descripcion': 'Bacheo', 'carga_reajustada': 1837.78}, {'codigo': 'MR-103', 'descripcion': 'Desquinche', 'carga_reajustada': 33.25}, {'codigo': 'MR-104', 'descripcion': 'Remoción de Derrumbes', 'carga_reajustada': 136.09}]}, {'codigo': 'MR-200', 'descripcion': 'LIMPIEZA DE OBRAS DE DRENAJE', 'subactividades': [{'codigo': 'MR-201', 'descripcion': 'Limpieza de Cunetas', 'carga_reajustada': 16666.67}, {'codigo': 'MR-203', 'descripcion': 'Limpieza de Badén', 'carga_reajustada': 466.67}]}, {'codigo': 'MR-300', 'descripcion': 'CONTROL DE VEGETACIÓN', 'subactividades': [{'codigo': 'MR-301', 'descripcion': 'Roce y Limpieza', 'carga_reajustada': 39671.67}]}, {'codigo': 'MR-400', 'descripcion': 'SEGURIDAD VIAL', 'subactividades': [{'codigo': 'MR-401', 'descripcion': 'Conservación de Señales'

In [98]:
with open("./data/"+proyecto + '/partidas.yaml', 'r', encoding='utf-8') as file:
    partidas = yaml.safe_load(file)

print(partidas)

[{'codigo': 'MR100', 'descripcion': 'CONSERVACION DE CALZADA', 'actividades': [{'codigo': 'MR101', 'descripcion': 'Limpieza de Calzada', 'unidad_medida': 'Km', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 45, 'rendimiento': 0.2}, {'codigo': 'MR102', 'descripcion': 'Bacheo', 'unidad_medida': 'm2', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 60, 'rendimiento': 10}, {'codigo': 'MR103', 'descripcion': 'Desquinche', 'unidad_medida': 'm3', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 5, 'rendimiento': 2.5}, {'codigo': 'MR104', 'descripcion': 'Remoción de Derrumbes', 'unidad_medida': 'm3', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 16, 'rendimiento': 3}]}, {'codigo': 'MR200', 'descripcion': 'LIMPIEZA DE OBRAS DE ARTE', 'actividades': [{'codigo': 'MR201', 'descripcion': 'Limpiez

## Funciones utiles

In [99]:
def concatenar_elementos(lista):
    elementos_como_texto = [str(e) for e in lista]
    return " ".join(elementos_como_texto)

In [100]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

In [101]:
from datetime import date

def mes_obra(fecha_inicio: date, mes: int) -> str:
    """
    Devuelve un string indicando el nº de mes de la obra respecto a la fecha de inicio.
    
    Args:
        fecha_inicio (date): Fecha en que empezó la obra (ej: date(2025,1,1)).
        mes (int): Número de mes (1–12) a evaluar.
    
    Returns:
        str: 
          - Si `mes` < mes de inicio: cadena vacía.
          - Si `mes` >= mes de inicio: "{n}º mes" donde n = (mes - mes_inicio + 1).
    """
    if not isinstance(fecha_inicio, date):
        raise TypeError("fecha_inicio debe ser un objeto date de datetime")
    if not (1 <= mes <= 12):
        raise ValueError("mes debe estar entre 1 y 12")
    
    # Calculamos cuántos meses han pasado desde el mes de inicio
    diff = mes - fecha_inicio.month + 1
    if diff < 1:
        return ""
    return f"{diff}º mes"


In [102]:
def nombre_mes(numero: int) -> str:
    """
    Devuelve el nombre del mes en español correspondiente al número dado.
    
    Parámetros:
        numero (int): Un entero entre 1 y 12.
        
    Retorna:
        str: Nombre del mes en español, o 'Mes inválido' si el número no está en rango.
    """
    meses = [
        None,       # índice 0 no usado
        "Enero",    # 1
        "Febrero",  # 2
        "Marzo",    # 3
        "Abril",    # 4
        "Mayo",     # 5
        "Junio",    # 6
        "Julio",    # 7
        "Agosto",   # 8
        "Septiembre", # 9
        "Octubre",  # 10
        "Noviembre",# 11
        "Diciembre" # 12
    ]
    
    if 1 <= numero <= 12:
        return meses[numero]
    else:
        return "Mes inválido"

In [103]:
def dias_mes_calculator(mes: int, anio: int) -> int:
    """
    Devuelve el número de días de un mes dado y año especificado.

    Parámetros:
    - mes (int): número del mes (1-12)
    - anio (int): año en formato YYYY

    Retorna:
    - int: cantidad de días en el mes
    """
    if not 1 <= mes <= 12:
        raise ValueError("El mes debe estar entre 1 y 12.")
    return calendar.monthrange(anio, mes)[1]



In [104]:
def letra_a_indice(columna):
    """
    Convierte una letra de columna Excel ('A', 'Z', 'AA', etc.) a su índice (1-based).
    """
    indice = 0
    for c in columna.upper():
        indice = indice * 26 + (ord(c) - ord('A') + 1)
    return indice

def indice_a_letra(indice):
    """
    Convierte un índice 1-based en su correspondiente letra de columna Excel.
    """
    letras = ''
    while indice > 0:
        indice, resto = divmod(indice-1, 26)
        letras = chr(resto + ord('A')) + letras
    return letras

def columna_final(inicio, dias):
    """
    Devuelve solo la letra de la columna donde termina,
    contando 'dias' columnas empezando en 'inicio'.
    
    Ejemplo: columna_final('F', 31) -> 'AJ'
    """
    idx_inicio = letra_a_indice(inicio)
    idx_fin    = idx_inicio + dias - 1
    return indice_a_letra(idx_fin)

# --- Ejemplo de uso ---
dias_en_mes = 31
fin = columna_final("F", dias_en_mes)  # -> "AJ"

In [105]:
def columna_a_indice(col):
    """Convierte letras de columna (p.ej. 'F', 'AA') a índice (1-based)."""
    idx = 0
    for letra in col:
        idx = idx * 26 + (ord(letra.upper()) - ord('A') + 1)
    return idx

def indice_a_columna(idx):
    """Convierte índice (1-based) a letras de columna."""
    letras = ''
    while idx:
        idx, resto = divmod(idx-1, 26)
        letras = chr(ord('A') + resto) + letras
    return letras

def generar_suma_intercalada(celda_inicio: str, numero_meses: int) -> str:
    """
    Dada una celda de inicio (p.ej. 'F6') y un número de meses,
    devuelve una cadena tipo '=F6+H6+J6+...'.
    Avanza 2 columnas por cada mes.
    """
    # Separa letras y número de fila
    m = re.match(r"^([A-Za-z]+)(\d+)$", celda_inicio)
    if not m:
        raise ValueError(f"Celda inválida: {celda_inicio}")
    col_letras, fila = m.groups()
    
    inicio_idx = columna_a_indice(col_letras)
    fila = fila  # sigue siendo string
    
    # Genera las referencias
    celdas = []
    for i in range(numero_meses):
        idx = inicio_idx + 2*i
        col = indice_a_columna(idx)
        celdas.append(f"{col}{fila}")
    
    return "=" + "+".join(celdas)

In [106]:
def split_month_into_four_index_blocks(year: int, month: int) -> List[Tuple[int, int]]:
    """
    Divide el mes en 4 bloques de días casi iguales y devuelve,
    para cada bloque, los índices de inicio y fin (0-based).
    Ejemplo: para un bloque que cubre días 1–8, devuelve (0, 7).
    """
    # 1) ¿Cuántos días tiene el mes?
    _, days_in_month = calendar.monthrange(year, month)
    # 2) División entera + resto
    q, r = divmod(days_in_month, 4)
    # 3) Longitudes de bloque: primeros r tienen q+1, resto q
    block_lengths = [q + 1 if i < r else q for i in range(4)]
    
    # 4) Construir lista de (start_idx, end_idx)
    indices: List[Tuple[int, int]] = []
    start_day = 1
    for length in block_lengths:
        start_idx = start_day - 1
        end_idx = start_idx + length - 1
        indices.append((start_idx, end_idx))
        start_day += length
    
    return indices

# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Para noviembre de 2025 (30 días → bloques de 8,8,7,7)
    print(split_month_into_four_index_blocks(2025, 11))
    # Salida: [(0, 7), (8, 15), (16, 22), (23, 29)]

[(0, 7), (8, 15), (16, 22), (23, 29)]


In [107]:
def extraer_subactividades(data: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """
    Dado un dict con la estructura:
      { "-actividades": [ { "codigo": ..., "subactividades": [ {...}, ... ] }, ... ] }
    devuelve un nuevo dict donde cada clave es el código de la subactividad
    (sin guiones) y el valor es el objeto completo de dicha subactividad.
    """
    resultado: Dict[str, Dict[str, Any]] = {}
    
    for actividad in data.get("-actividades", []):
        for sub in actividad.get("subactividades", []):
            codigo = sub.get("codigo")
            if codigo:
                # Eliminamos los guiones del código para usarlo como key
                clave_sin_guion = codigo.replace("-", "")
                resultado[clave_sin_guion] = sub

    return resultado

In [108]:
def calcular_pagos_por_mes(
    fecha_inicio: date,
    duracion: int,
    monto: float
) -> Dict[int, float]:
    """
    Distribuye el pago total `monto` proporcionalmente según los días de obra
    que caen en cada mes, a partir de `fecha_inicio` y durante `duracion` días.

    Retorna:
        Un diccionario {mes: costo}, donde mes está en 1..12.
    """
    if duracion <= 0:
        raise ValueError("La duración debe ser un entero positivo.")
    
    # Fecha de fin del proyecto (inclusive)
    fecha_fin = fecha_inicio + timedelta(days=duracion - 1)
    
    # Contar días de obra en cada mes
    dias_por_mes = {}  # { (año, mes): días }
    año, mes = fecha_inicio.year, fecha_inicio.month
    
    while (año, mes) <= (fecha_fin.year, fecha_fin.month):
        # Primer y último día del mes actual
        primer_dia_mes = date(año, mes, 1)
        ultimo_dia_mes = date(año, mes, calendar.monthrange(año, mes)[1])
        
        # Rango efectivo dentro del mes
        inicio_effectivo = max(fecha_inicio, primer_dia_mes)
        fin_effectivo    = min(fecha_fin, ultimo_dia_mes)
        
        dias = (fin_effectivo - inicio_effectivo).days + 1
        dias_por_mes[(año, mes)] = dias
        
        # Avanzar al siguiente mes
        if mes == 12:
            año += 1
            mes = 1
        else:
            mes += 1
    
    # Calcular proporciones y costos
    pagos = {}
    # Total de días (debería coincidir con `duracion`)
    total_dias = sum(dias_por_mes.values())
    
    # Para asegurar suma exacta tras redondeos
    meses = list(dias_por_mes.keys())
    pagos_temp = []
    for (a, m), dias in dias_por_mes.items():
        proporción = dias / total_dias
        costo_raw = monto * proporción
        pagos_temp.append(((a, m), costo_raw))
    
    # Redondear todos menos el último, y ajustar el último
    pagos_rounded = {}
    acumulado = 0.0
    for (a, m), costo_raw in pagos_temp[:-1]:
        costo = round(costo_raw, 2)
        pagos_rounded[(a, m)] = costo
        acumulado += costo
    
    # Último mes
    último_mes = pagos_temp[-1][0]
    pagos_rounded[último_mes] = round(monto - acumulado, 2)
    
    # Devolver con clave solo el número de mes
    # (si el proyecto cruza años, podrían sobrescribirse; en ese caso
    # convendría usar (año, mes) como clave)
    resultado = {m: costo for (a, m), costo in pagos_rounded.items()}
    return resultado

In [109]:
def contar_meses_obra(fecha_inicio: date, plazo_dias: int) -> int:
    """
    Calcula el número de meses distintos en los que se realiza trabajo,
    dado:
      - fecha_inicio: fecha de inicio de la obra (incluida)
      - plazo_dias: duración de la obra en días naturales

    Se considera que, si la obra entra siquiera un día en un mes,
    ese mes cuenta entero.

    Devuelve un entero con el número de meses abarcados.
    """
    if plazo_dias <= 0:
        raise ValueError("El plazo debe ser un entero positivo")

    # Fecha del último día de trabajo
    fecha_fin = fecha_inicio + timedelta(days=plazo_dias - 1)

    # Cálculo de diferencia de meses (incluyendo ambos extremos)
    meses = (
        (fecha_fin.year - fecha_inicio.year) * 12
        + (fecha_fin.month - fecha_inicio.month)
        + 1
    )
    return meses

## Valores calculados relevantes

In [110]:
fecha_inicio=date(datos['fecha_inicio']['year'], datos['fecha_inicio']['month'], datos['fecha_inicio']['day'])
fecha_fin=fecha_inicio + timedelta(days=datos['tiempo_ejecucion_dias']-1) #Calculamos la fecha final sumando duracion_dias - 1 días (porque si el día 1 ya cuenta como primer día, solo sumamos los días restantes).
print("Fecha de inicio:", fecha_inicio)
print("Fecha de fin:", fecha_fin)

Fecha de inicio: 2025-04-10
Fecha de fin: 2025-12-05


## Programacion

In [111]:
h1=concatenar_elementos([
    "Programación inicial base de actividades",
    datos['fecha_inicio']['year'],
    "-",
    datos['tiempo_ejecucion_dias'],
    "días calendarios"]
)
print(h1)

Programación inicial base de actividades 2025 - 240 días calendarios


In [112]:
header_ruta=concatenar_elementos([
    'Ruta:',
    datos['ruta']])
header_tramo=concatenar_elementos([
    'Tramo:',
    datos['tramo']])
header_longitud=concatenar_elementos([
    'Longitud:',
    formatear_progresiva(datos['longitud']),
    "Km"]
    )

header_fecha_inicio=concatenar_elementos([
    'Fecha de inicio:',
    fecha_inicio.strftime("%d/%m/%Y")])
header_fecha_fin=concatenar_elementos([
    'Fecha de fin:',
    fecha_fin.strftime("%d/%m/%Y")])
header_tiempo_ejecucion=concatenar_elementos([
    'Tiempo de ejecución:',
    datos['tiempo_ejecucion_dias'],
    "días calendarios"])

In [113]:
print(header_fecha_fin)
print(header_fecha_inicio)

Fecha de fin: 05/12/2025
Fecha de inicio: 10/04/2025


In [114]:
# Actualización del script para incluir actividades con 0 días en la tabla final

# 1) PARÁMETROS DE ENTRADA
# -----------------------
json_actividades = partidas


# 2) EXTRACCIÓN DE ACTIVIDADES
# -----------------------------
actividades_all = {}
for grupo in json_actividades:
    for act in grupo["actividades"]:
        actividades_all[act["codigo"]] = act["numero_dias"]

# Separar actividades con >0 y aquellas con 0
actividades = {i: d for i, d in actividades_all.items() if d > 0}
actividades_zero = [i for i, d in actividades_all.items() if d == 0]

# 3) VALIDACIÓN DE CONSISTENCIA
# -----------------------------
total_act = sum(actividades.values())
total_periodo = (fecha_fin - fecha_inicio).days + 1
if total_act != total_periodo:
    raise RuntimeError(f"Error: suma de actividades ({total_act}) ≠ días del periodo ({total_periodo})")

# 4) GENERAR CALENDARIO POR MES
# -----------------------------
calendario = {}
hoy = fecha_inicio
while hoy <= fecha_fin:
    mes = hoy.strftime("%Y-%m")
    calendario[mes] = calendario.get(mes, 0) + 1
    hoy += timedelta(days=1)

# 5) MATRIZ IDEAL r[i][j]
# -----------------------
meses = list(calendario.keys())
S = sum(calendario.values())
r = {i: {m: actividades[i] * (calendario[m] / S) for m in meses} for i in actividades}

# 6) MODELO MIP CON PuLP
# -----------------------
prob = pulp.LpProblem("DistribucionDias", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", (actividades.keys(), meses), lowBound=0, cat="Integer")
d = pulp.LpVariable.dicts("d", (actividades.keys(), meses), lowBound=0, cat="Continuous")

# Restricciones por actividad
for row, D_i in actividades.items():
    prob += pulp.lpSum(x[row][m] for m in meses) == D_i, f"fila_{row}"

# Restricciones por mes
for m, M_m in calendario.items():
    prob += pulp.lpSum(x[i][m] for i in actividades) == M_m, f"col_{m}"

# Desviaciones absolutas
for row in actividades:
    for m in meses:
        prob += x[row][m] - r[row][m] <= d[row][m]
        prob += r[row][m] - x[row][m] <= d[row][m]

# Objetivo
prob += pulp.lpSum(d[i][m] for i in actividades for m in meses)

# Resolver
print("Resolviendo MIP…")
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# 7) EXTRAER SOLUCIÓN
data = {i: {m: int(pulp.value(x[i][m])) for m in meses} for i in actividades}

# Agregar actividades con 0 días
for row in actividades_zero:
    data[row] = {m: 0 for m in meses}

# 8) Mostrar tabla
# 1) Crear DataFrame y reordenar columnas cronológicamente
monthly_activities_summary_df = pd.DataFrame(data).T
meses = sorted(calendario.keys())      # ej. ['2025-05', '2025-06', …, '2025-12']
monthly_activities_summary_df = monthly_activities_summary_df[meses]

# 2) Total por actividad (filas)
monthly_activities_summary_df["TOTAL"] = monthly_activities_summary_df.sum(axis=1).astype(int)

# 3) Total por mes (columnas), incluyendo la columna "TOTAL"
column_totals = monthly_activities_summary_df.sum(axis=0).astype(int)
monthly_activities_summary_df.loc["TOTAL"] = column_totals

# 4) (Opcional) Reordenar filas para un orden específico
#    p.ej., manteniendo el orden original de actividades y luego "TOTAL"
orden_filas = list(actividades_all.keys()) + ["TOTAL"]
monthly_activities_summary_df = monthly_activities_summary_df.reindex(orden_filas)

print(monthly_activities_summary_df)



Resolviendo MIP…
       2025-04  2025-05  2025-06  2025-07  2025-08  2025-09  2025-10  2025-11  \
MR101        4        6        5        6        6        6        6        5   
MR102        5        8        7        8        8        7        8        8   
MR103        0        0        1        1        1        1        0        1   
MR104        2        2        2        2        2        2        2        2   
MR201        4        6        5        6        6        5        6        5   
MR202        0        0        0        0        0        0        0        0   
MR203        1        1        2        2        1        1        1        1   
MR204        0        0        0        0        0        0        0        0   
MR205        0        0        0        0        0        0        0        0   
MR206        0        0        0        0        0        0        0        0   
MR301        4        6        5        6        6        5        6        5   
MR401      

In [115]:
monthly_activities_summary_df.head(18)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,4,6,5,6,6,6,6,5,1,45
MR102,5,8,7,8,8,7,8,8,1,60
MR103,0,0,1,1,1,1,0,1,0,5
MR104,2,2,2,2,2,2,2,2,0,16
MR201,4,6,5,6,6,5,6,5,1,44
MR202,0,0,0,0,0,0,0,0,0,0
MR203,1,1,2,2,1,1,1,1,1,11
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0
MR206,0,0,0,0,0,0,0,0,0,0


In [116]:
full_months = pd.period_range('2025-01', '2025-12', freq='M').strftime('%Y-%m').tolist()


In [117]:
df_full   = monthly_activities_summary_df.reindex(columns=full_months, fill_value=0)

### Escribiendo el documento excel

In [118]:
# 1. Crear el libro y la hoja
folder_path = "output"
file_path = os.path.join(folder_path, f"{proyecto}.xlsx")

wb = xlsxwriter.Workbook(file_path)
ws = wb.add_worksheet("programacion")


# Crear la carpeta si no existe
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 2. Definir formatos
header_fmt = wb.add_format(
    {
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": "#D9E1F2",
        "border": 1,
    }
)

header2_fmt = wb.add_format(
    {
        "bold": True,
        "align": "left",
        "valign": "vcenter",
    }
)

header_label_fmt = wb.add_format(
    {
        "bold": True,
        "align": "center",
        "valign": "vcenter",
    }
)

table_header_fmt = wb.add_format(
    {
        "align": "center",
        "valign": "vcenter",
        "bg_color": "#C5D9F1",
        "border": 1,
        "text_wrap": True,
        "shrink": True,
    }
)

table_header_number_integer_fmt = wb.add_format(
    {
        "align": "center",
        "valign": "vcenter",
        "bg_color": "#C5D9F1",
        "border": 1,
    }
)


cell_fmt = wb.add_format(
    {"valign": "vcenter", "border": 1, "font_size": 8, "num_format": "#,##0.00"}
)

cell_text_break_fmt = wb.add_format(
    {
        "align": "left",
        "valign": "vcenter",
        "text_wrap": True,
        "shrink": True,
        "border": 1,
    }
)

cell_align_right_fmt = wb.add_format(
    {
        "align": "right",
        "valign": "vcenter",
        "border": 1,
    }
)


cell_number_fmt = wb.add_format(
    {
        "valign": "vcenter",
        "border": 1,
        "font_size": 8,
        "num_format": "#,##0.00",
    }
)


suma_fmt = wb.add_format(
    {
        "valign": "vcenter",
        "border": 1,
        "num_format": "#,##0.00",
    }
)


soles_color_fmt = wb.add_format(
    {
        "valign": "vcenter",
        "border": 1,
        "bold": True,
        "bg_color": "#D9E1F2",
        "num_format": '"S/." #,##0.00',
    }
)

soles_fmt = wb.add_format(
    {"valign": "vcenter", "border": 1, "num_format": '"S/." #,##0.00'}
)

formato_ajustar = wb.add_format(
    {
        "align": "center",
        "valign": "vcenter",
        "text_wrap": True,  # Ajustar texto
        "shrink": True,  # Reducir hasta ajustar
        "border": 1,
    }
)


month_fmt = wb.add_format({"align": "center", "bg_color": "#F2F2F2", "border": 1})
code_fmt = wb.add_format({"border": 1})
number_fmt = wb.add_format({"border": 1, "align": "center"})

# 3. Ajustar anchos de columna
ws.set_column("B:B", 8)  # Código
ws.set_column("C:C", 30)  # Actividad
ws.set_column("D:H", 8)  # Unid., Tab./Cuad., etc.
ws.set_column("H:T", 8)  # Meses
ws.set_column("U:U", 10)  # Nº días

# 4. Escribir título y combinar celdas
ws.merge_range("B2:U2", h1, header_fmt)

ws.merge_range("C3:J3", header_ruta, header2_fmt)
ws.merge_range("C4:J4", header_tramo, header2_fmt)
ws.merge_range("C5:J5", header_longitud, header2_fmt)

ws.merge_range("K3:O3", header_fecha_inicio, header2_fmt)
ws.merge_range("K4:O4", header_fecha_fin, header2_fmt)
ws.merge_range("K5:O5", header_tiempo_ejecucion, header2_fmt)

# 5. Escribir encabezados de tabla

ws.merge_range("B7:B10", "Código", table_header_fmt)
ws.merge_range("C7:C10", "Actividad", table_header_fmt)
ws.merge_range("D7:D10", "Unid.", table_header_fmt)
ws.merge_range("E7:E10", "Tab./Cuad.", table_header_fmt)
ws.merge_range("F7:F10", "Nº Cuad.", table_header_fmt)
ws.merge_range("G7:G10", "Tab. / Act.", table_header_fmt)
ws.merge_range("H7:H10", "Nº Dias", table_header_fmt)

ws.merge_range("I7:T7", "Meses", table_header_fmt)
ws.write("I8", "ENE", table_header_fmt)
ws.write("J8", "FEB", table_header_fmt)
ws.write("K8", "MAR", table_header_fmt)
ws.write("L8", "ABR", table_header_fmt)
ws.write("M8", "MAY", table_header_fmt)
ws.write("N8", "JUN", table_header_fmt)
ws.write("O8", "JUL", table_header_fmt)
ws.write("P8", "AGO", table_header_fmt)
ws.write("Q8", "SEP", table_header_fmt)
ws.write("R8", "OCT", table_header_fmt)
ws.write("S8", "NOV", table_header_fmt)
ws.write("T8", "DIC", table_header_fmt)

ws.merge_range("I9:K9", "ÉPOCA DE LLUVIAS", table_header_fmt)
ws.merge_range("L9:M9", "DESPUES DE LLUVIAS", table_header_fmt)
ws.merge_range("N9:Q9", "ÉPOCA SECA", table_header_fmt)
ws.merge_range("R9:T9", "ANTES DE LLUVIAS", table_header_fmt)

ws.merge_range("U7:U10", "N° días para ejecutar según contrato", formato_ajustar)

# VALORES CALCULADOS

ws.write("I10", mes_obra(fecha_inicio, 1), table_header_fmt)
ws.write("J10", mes_obra(fecha_inicio, 2), table_header_fmt)
ws.write("K10", mes_obra(fecha_inicio, 3), table_header_fmt)
ws.write("L10", mes_obra(fecha_inicio, 4), table_header_fmt)
ws.write("M10", mes_obra(fecha_inicio, 5), table_header_fmt)
ws.write("N10", mes_obra(fecha_inicio, 6), table_header_fmt)
ws.write("O10", mes_obra(fecha_inicio, 7), table_header_fmt)
ws.write("P10", mes_obra(fecha_inicio, 8), table_header_fmt)
ws.write("Q10", mes_obra(fecha_inicio, 9), table_header_fmt)
ws.write("R10", mes_obra(fecha_inicio, 10), table_header_fmt)
ws.write("S10", mes_obra(fecha_inicio, 11), table_header_fmt)
ws.write("T10", mes_obra(fecha_inicio, 12), table_header_fmt)

# 2. Prepara las filas
rows = []
for sección in partidas:
    # fila de sección (solo código y descripción, resto vacío)
    rows.append(
        {
            "Código": sección["codigo"],
            "Actividad": sección["descripcion"],
            "Unid.": "",
            "Tab. / Cuad.": "",
            "Nº Cuad.": "",
            "Tab. / Act.": "",
            "Nº Días": "",
        }
    )
    # filas de actividades
    for act in sección["actividades"]:
        rows.append(
            {
                "Código": act["codigo"],
                "Actividad": act["descripcion"],
                "Unid.": act["unidad_medida"],
                "Tab. / Cuad.": act["tablas_por_cuadrilla"],
                "Nº Cuad.": act["numero_cuadrillas"],
                "Tab. / Act.": act["total_tablas_por_actividad"],
                "Nº Días": act["numero_dias"],
            }
        )

# 8. Guardar el archivo

# 1. Define formatos
section_fmt = wb.add_format(
    {
        "bold": True,
        "border": 1,
    }
)

no_section_fmt = wb.add_format(
    {
        "border": 1,
    }
)
activity_indent = wb.add_format({"indent": 1, "border": 1})

# 2. Punto de inicio en B11
start_row, start_col = 10, 1  # B11 en 0‑based

# 3. Recorre rows y escribe
for i, row in enumerate(rows):
    r = start_row + i
    # si es fila de sección (unidad vacía), negrita; si no, formato normal
    is_seccion = row["Unid."] == ""
    fmt = section_fmt if is_seccion else None

    # columna B → Código
    ws.write(r, start_col + 0, row["Código"], fmt)
    # columna C → Actividad (con sangría si no es sección)
    ws.write(
        r,
        start_col + 1,
        row["Actividad"],
        section_fmt if is_seccion else activity_indent,
    )
    # resto de columnas D‑H
    ws.write(r, start_col + 2, row["Unid."], fmt)
    ws.write(r, start_col + 3, row["Tab. / Cuad."], fmt)
    ws.write(r, start_col + 4, row["Nº Cuad."], fmt)
    ws.write(r, start_col + 5, row["Tab. / Act."], fmt)
    ws.write(r, start_col + 6, row["Nº Días"], fmt)

# 4. (Opcional) Ajusta anchos
ws.set_column(start_col + 0, start_col + 0, 12)  # Código
ws.set_column(start_col + 1, start_col + 1, 40)  # Actividad
ws.set_column(start_col + 2, start_col + 6, 12)  # resto

## Llenar la tabla con los datos de df_full

# 2) Lista de códigos en el orden de tu hoja (incluye títulos y detalles)
sheet_codes = [
    "MR100",  # título
    "MR101",
    "MR102",
    "MR103",
    "MR104",
    "MR200",  # otro título
    "MR201",
    "MR202",
    "MR203",
    "MR204",
    "MR205",
    "MR206",
    "MR300",  # otro título
    "MR301",
    "MR400",  # otro título
    "MR401",
    "MR500",  # otro título
    "MR501",
    "MR600",  # otro título
    "MR601",
    "MR700",  # otro título
    "MR701",
    "MR702",
]

start_row = 11  # fila donde empiezan (1-based)
start_col = 9  # col I → 9 (1-based)
for i, code in enumerate(sheet_codes):
    row = start_row - 1 + i  # 0-index
    if code in df_full.index:
        for j, mes in enumerate(full_months):
            col = start_col - 1 + j
            val = df_full.at[code, mes]
            ws.write(row, col, val)
    # si no está en df_full (fila de título), no escribimos nada → queda en blanco

# LLENANDO LA TABLA CON LOS TOTALES

filas = [12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 24, 26, 28, 30, 32, 33]

# 3. Bucle para escribir la fórmula SUM(I#:T#) en la columna U de cada fila
for fila in filas:
    celda_destino = f"U{fila}"  # e.g. 'U12', 'U13', …
    formula = f"=SUM(I{fila}:T{fila})"  # e.g. '=SUM(I12:T12)'
    ws.write_formula(celda_destino, formula, suma_fmt)

# 2. Generar la lista de columnas de la 'I' a la 'U'
#    Usamos los códigos ASCII para pasar de 'I' (73) a 'U' (85).
columnas = [chr(c) for c in range(ord("I"), ord("U") + 1)]

# 3. Bucle para insertar SUMA en fila 34 de cada columna
for col in columnas:
    celda_destino = f"{col}34"  # e.g. 'I34', 'J34', …
    rango = f"{col}12:{col}33"  # e.g. 'I12:I33'
    formula = f"=SUM({rango})"  # '=SUM(I12:I33)'
    ws.write_formula(celda_destino, formula, suma_fmt)

# AGREGANDO BORDE DE LA TABLA

ws.conditional_format("B11:U33", {"type": "no_errors", "format": cell_fmt})

# wb.close()

0

## Numero de dias - excel

In [119]:
subactividades=extraer_subactividades(carga_trabajo)
print(subactividades)

{'MR101': {'codigo': 'MR-101', 'descripcion': 'Limpieza de Calzada', 'carga_reajustada': 17.33}, 'MR102': {'codigo': 'MR-102', 'descripcion': 'Bacheo', 'carga_reajustada': 1837.78}, 'MR103': {'codigo': 'MR-103', 'descripcion': 'Desquinche', 'carga_reajustada': 33.25}, 'MR104': {'codigo': 'MR-104', 'descripcion': 'Remoción de Derrumbes', 'carga_reajustada': 136.09}, 'MR201': {'codigo': 'MR-201', 'descripcion': 'Limpieza de Cunetas', 'carga_reajustada': 16666.67}, 'MR203': {'codigo': 'MR-203', 'descripcion': 'Limpieza de Badén', 'carga_reajustada': 466.67}, 'MR301': {'codigo': 'MR-301', 'descripcion': 'Roce y Limpieza', 'carga_reajustada': 39671.67}, 'MR401': {'codigo': 'MR-401', 'descripcion': 'Conservación de Señales', 'carga_reajustada': 29.33}, 'MR601': {'codigo': 'MR-601', 'descripcion': 'Vigilancia y Control', 'carga_reajustada': 112.67}, 'MR701': {'codigo': 'MR-701', 'descripcion': 'Reparación de Muros Secos', 'carga_reajustada': 19.33}}


In [120]:
h1 = concatenar_elementos([
    'Cronograma de ejecución de actividades'])

header_servicio = concatenar_elementos([
    'Servicio:',
    datos['servicio']])

header_contrato = concatenar_elementos([
    'Tramo:',
    datos['tramo']])

header_contratista = concatenar_elementos([
    'Longitud:',
    formatear_progresiva(datos['longitud'])])

header_plazo_ejecucion = concatenar_elementos([
    'Plazo de ejecución:',
    datos['tiempo_ejecucion_dias'],
    "días calendarios"])

header_numero_cuadrillas = concatenar_elementos([
    'Número de cuadrillas:',
    datos['numero_cuadrillas']])

header_numero_trabajadores=concatenar_elementos([
    'Número de trabajadores:',
    datos['numero_trabajadores']])

In [121]:
ws_num_dias = wb.add_worksheet("num_dias")

# Ajustar anchos de columna
ws_num_dias.set_column("A:A", 10)  # Código
ws_num_dias.set_column("B:B", 40)  # Actividades
ws_num_dias.set_column("C:C", 10)  # Unidad
ws_num_dias.set_column("D:I", 12)  # Unid., Tab./Cuad., etc.


ws_num_dias.merge_range("A1:J1", h1, header_fmt)

ws_num_dias.merge_range("A3:G3", header_servicio, header2_fmt)
ws_num_dias.merge_range("A4:G4", header_contrato, header2_fmt)
ws_num_dias.merge_range("A5:G5", header_contratista, header2_fmt)
ws_num_dias.merge_range("A6:G6", header_plazo_ejecucion, header2_fmt)
ws_num_dias.merge_range("A7:G7", header_numero_cuadrillas, header2_fmt)
ws_num_dias.merge_range("A8:G8", header_numero_trabajadores, header2_fmt)

ws_num_dias.set_row(9, 60)  # Ajustar altura de la fila 10

# Escribir encabezados de tabla

ws_num_dias.write("A10", "Código", table_header_fmt)
ws_num_dias.write("B10", "Actividad", table_header_fmt) 
ws_num_dias.write("C10", "Unidad", table_header_fmt)
ws_num_dias.write("D10", "Rend. Unitario", table_header_fmt)
ws_num_dias.write("E10", "Rend. Diario por cuadrilla", table_header_fmt)
ws_num_dias.write("F10", "Carga anual", table_header_fmt)
ws_num_dias.write("G10", "Carga 8 meses", table_header_fmt)
ws_num_dias.write("H10", "N.º de dias para ejecutar una actividad", table_header_fmt)
ws_num_dias.write("I10", "N.º de días para ejecutar según contrato", table_header_fmt)

rows=[]

for sección in partidas:

    # fila de sección (solo código y descripción, resto vacío)
    rows.append(
        {
            "Código": sección["codigo"],
            "Actividad": sección["descripcion"],
            "Unidad": "",
            "Rend. Unitario": "",
            "Rend. Diario por cuadrilla": "",
            "Carga anual": "",
            "Carga 8 meses": "",
            "N.º de días para ejecutar una actividad": "",
            "N.º de días para ejecutar según contrato": "",
        }
    )
    # filas de actividades
    for act in sección["actividades"]:
        if act['codigo']  in subactividades:
            calculated_carga_rejustada = subactividades[act['codigo']]['carga_reajustada']
            calculated_num_days_for_activity =calculated_carga_rejustada/(act["rendimiento"]*datos['numero_trabajadores'])

            print(calculated_num_days_for_activity)

            calculated_num_days_for_activity_contract=monthly_activities_summary_df.at[act['codigo'], 'TOTAL']

            print(calculated_num_days_for_activity_contract)

            rows.append(
                {
                    "Código": act["codigo"],
                    "Actividad": act["descripcion"],
                    "Unidad": act["unidad_medida"],
                    "Rend. Unitario": act["rendimiento"],
                    "Rend. Diario por cuadrilla": act["rendimiento"]*datos['numero_trabajadores'],
                    "Carga anual": calculated_carga_rejustada*12/8,
                    "Carga 8 meses": calculated_carga_rejustada,
                    "N.º de días para ejecutar una actividad": calculated_num_days_for_activity,
                    "N.º de días para ejecutar según contrato": calculated_num_days_for_activity_contract
                }
            )

        else:
            rows.append(
                {
                    "Código": act["codigo"],
                    "Actividad": act["descripcion"],
                    "Unidad": act["unidad_medida"],
                    "Rend. Unitario": act["rendimiento"],
                    "Rend. Diario por cuadrilla": act["rendimiento"]*datos['numero_trabajadores'],
                    "Carga anual": 0,
                    "Carga 8 meses": 0,
                    "N.º de días para ejecutar una actividad": 0,
                    "N.º de días para ejecutar según contrato": 0
                }
            )

start_row, start_col = 10, 0 # B11 en 0‑based
for i, row in enumerate(rows):
    r = start_row + i
    # si es fila de sección (unidad vacía), negrita; si no, formato normal
    is_seccion = row["Unidad"] == ""
    fmt = section_fmt if is_seccion else no_section_fmt

    # columna B → Código
    ws_num_dias.write(r, start_col + 0, row["Código"], fmt)
    # columna C → Actividad (con sangría si no es sección)
    ws_num_dias.write(
        r,
        start_col + 1,
        row["Actividad"],
        section_fmt if is_seccion else activity_indent,
    )
    # resto de columnas D‑H
    ws_num_dias.write(r, start_col + 2, row["Unidad"], fmt)
    ws_num_dias.write(r, start_col + 3, row["Rend. Unitario"], fmt)
    ws_num_dias.write(r, start_col + 4, row["Rend. Diario por cuadrilla"], fmt)
    ws_num_dias.write(r, start_col + 5, row["Carga anual"], fmt)
    ws_num_dias.write(r, start_col + 6, row["Carga 8 meses"], fmt)
    ws_num_dias.write(r, start_col + 7, row["N.º de días para ejecutar una actividad"], fmt)
    ws_num_dias.write(r, start_col + 8, row["N.º de días para ejecutar según contrato"], fmt)



21.662499999999998
45
45.9445
60
3.325
5
11.340833333333334
16
34.722229166666665
44
11.66675
11
24.79479375
44
0.14665
5
1.1267
5
4.027083333333334
5


## Suposicion del resumen

In [122]:
# Construir un diccionario de cargas por código (sin guion, coincidiendo con el índice del df)
carga_dict = {}
for act in carga_trabajo["-actividades"]:
    for sub in act["subactividades"]:
        key = sub["codigo"].replace("-", "")  # "MR-101" -> "MR101"
        carga_dict[key] = sub["carga_reajustada"]

carga_series = pd.Series(carga_dict)

# Seleccionar columnas de meses
months = [c for c in monthly_activities_summary_df.columns if c != "TOTAL"]

# 2) Preparamos el denominador: 0 → NaN
total_days = monthly_activities_summary_df["TOTAL"].replace({0: np.nan})

# 3) Calculamos la carga proporcional y sustituimos NaN por 0
load_df = (
    monthly_activities_summary_df[months]
      .div(total_days, axis=0)         # división segura
      .mul(carga_series, axis=0)       # multiplicación por carga ajustada
).fillna(0)                            # todo NaN → 0

# 4) Reconstruimos la columna TOTAL (suma de cargas mensuales)
load_df["TOTAL"] = load_df[months].sum(axis=1)

# Mostrar resultado
load_df

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,1.540444,2.310667,1.925556,2.310667,2.310667,2.310667,2.310667,1.925556,0.385111,17.33
MR102,153.148333,245.037333,214.407667,245.037333,245.037333,214.407667,245.037333,245.037333,30.629667,1837.78
MR103,0.000000,0.000000,6.650000,6.650000,6.650000,6.650000,0.000000,6.650000,0.000000,33.25
MR104,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,0.000000,136.09
MR201,1515.151818,2272.727727,1893.939773,2272.727727,2272.727727,1893.939773,2272.727727,1893.939773,378.787955,16666.67
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,42.424545,42.424545,84.849091,84.849091,42.424545,42.424545,42.424545,42.424545,42.424545,466.67
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [123]:
load_df.index.name = "codigo"
monthly_activities_summary_df.index.name = "codigo"

In [124]:
monthly_activities_summary_df.head(20)


,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR101,4,6,5,6,6,6,6,5,1,45
MR102,5,8,7,8,8,7,8,8,1,60
MR103,0,0,1,1,1,1,0,1,0,5
MR104,2,2,2,2,2,2,2,2,0,16
MR201,4,6,5,6,6,5,6,5,1,44
MR202,0,0,0,0,0,0,0,0,0,0
MR203,1,1,2,2,1,1,1,1,1,11
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0


In [125]:
load_df.head(20)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR101,1.540444,2.310667,1.925556,2.310667,2.310667,2.310667,2.310667,1.925556,0.385111,17.33
MR102,153.148333,245.037333,214.407667,245.037333,245.037333,214.407667,245.037333,245.037333,30.629667,1837.78
MR103,0.000000,0.000000,6.650000,6.650000,6.650000,6.650000,0.000000,6.650000,0.000000,33.25
MR104,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,0.000000,136.09
MR201,1515.151818,2272.727727,1893.939773,2272.727727,2272.727727,1893.939773,2272.727727,1893.939773,378.787955,16666.67
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,42.424545,42.424545,84.849091,84.849091,42.424545,42.424545,42.424545,42.424545,42.424545,466.67
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [126]:
monthly_activities_summary_df.columns

Index(['2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09',
       '2025-10', '2025-11', '2025-12', 'TOTAL'],
      dtype='object')

In [127]:
# Lista de meses (excluyendo TOTAL)
meses = [col for col in monthly_activities_summary_df.columns if col != "TOTAL"]

# Diccionario para almacenar los DataFrames por mes
dfs_por_mes = {}

for mes in meses:
    # Crear el DataFrame del mes con 'codigo', 'dias', 'carga'
    df_mes = pd.DataFrame({
        'codigo': monthly_activities_summary_df.index,
        'dias': monthly_activities_summary_df[mes],
        'carga': load_df[mes]
    })
    
    # Calcular la carga por día (evitando división por cero)
    df_mes['carga_por_dia'] = df_mes.apply(
        lambda row: row['carga'] / row['dias'] if row['dias'] != 0 else 0, axis=1
    )
    
    # Guardar el DataFrame en el diccionario
    dfs_por_mes[mes] = df_mes

# Ejemplo: ver "2025-05"
dfs_por_mes["2025-05"]

,codigo,dias,carga,carga_por_dia
codigo,,,,
MR101,MR101,6,2.310667,0.385111
MR102,MR102,8,245.037333,30.629667
MR103,MR103,0,0.000000,0.000000
MR104,MR104,2,17.011250,8.505625
MR201,MR201,6,2272.727727,378.787955
MR202,MR202,0,0.000000,0.000000
MR203,MR203,1,42.424545,42.424545
MR204,MR204,0,0.000000,0.000000
MR205,MR205,0,0.000000,0.000000


In [128]:
dfs_por_mes_copy = {clave: df.copy(deep=True) for clave, df in dfs_por_mes.items()}

indices_a_eliminar = ["MR601", "TOTAL"]

for nombre_df, df in dfs_por_mes_copy.items():
    dfs_por_mes_copy[nombre_df] = df.drop(index=indices_a_eliminar, errors='ignore')

dfs_dict_filtrado = {
    key: df.loc[df['dias'] != 0].copy()
    for key, df in dfs_por_mes_copy.items()
}

dfs_dict_filtrado["2025-05"].head(20)


,codigo,dias,carga,carga_por_dia
codigo,,,,
MR101,MR101,6,2.310667,0.385111
MR102,MR102,8,245.037333,30.629667
MR104,MR104,2,17.011250,8.505625
MR201,MR201,6,2272.727727,378.787955
MR203,MR203,1,42.424545,42.424545
MR301,MR301,6,5409.773182,901.628864
MR401,MR401,1,5.866000,5.866000
MR701,MR701,1,3.866000,3.866000


In [129]:
dias_dict = dfs_por_mes_copy['2025-05']['dias'].to_dict()
print(dias_dict)

{'MR101': 6, 'MR102': 8, 'MR103': 0, 'MR104': 2, 'MR201': 6, 'MR202': 0, 'MR203': 1, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 6, 'MR401': 1, 'MR501': 0, 'MR701': 1, 'MR702': 0}


In [130]:
holidays_2025 = [
    date(2025, 5, 2),   # Día no laborable para el sector público
    date(2025, 6, 7),   # Batalla de Arica y Día de la Bandera
    date(2025, 6, 29),  # Día de San Pedro y San Pablo
    date(2025, 7, 23),  # Día de la Fuerza Aérea del Perú
    date(2025, 7, 28),  # Fiestas Patrias
    date(2025, 7, 29),  # Fiestas Patrias
    date(2025, 8, 6),   # Batalla de Junín
    date(2025, 8, 30),  # Santa Rosa de Lima
    date(2025, 10, 8),  # Combate de Angamos
    date(2025, 11, 1),  # Día de Todos los Santos
    date(2025, 12, 8),  # Inmaculada Concepción
    date(2025, 12, 9),  # Batalla de Ayacucho
    date(2025, 12, 25), # Navidad
    date(2025, 12, 26), # Día no laborable para el sector público
    #date(2025, 1, 1),   # Año Nuevo
    #date(2025, 1, 2),   # Día no laborable para el sector público
]

## funcion para el distribucion de actividades

In [131]:
def distribuir_actividades_ilp(activities, year, month, holidays):
    """
    Distribuye las actividades de manera uniforme en los días hábiles de un mes,
    respetando continuidad de bloques y evitando fines de semana y feriados.

    Parámetros:
    - activities: dict actividad→número de días a asignar.
    - year: año (int), p.ej. 2025.
    - month: mes (int, 1–12).
    - holidays: lista de objetos datetime.date con los feriados del mes.

    Retorna:
    - pandas.DataFrame con índice actividades y columnas fechas (YYYY-MM-DD),
      valores 1 si la actividad se realiza ese día, 0 en caso contrario.
    """
    # 1. Generar lista de días hábiles
    start_date = date(year, month, 1)
    end_date = (pd.Timestamp(start_date) + pd.offsets.MonthEnd(0)).date()
    all_days = pd.date_range(start=start_date, end=end_date, freq='D')
    business_days = [
        d.date() for d in all_days
        if d.weekday() < 5 and d.date() not in holidays
    ]
    B = len(business_days)

    # 2. Modelo ILP
    model = pulp.LpProblem("Distribucion_Actividades", pulp.LpMinimize)

    # Variables de inicio (s) y asignación (x)
    s = {
        (i, j): pulp.LpVariable(f"s_{i}_{j}", cat='Binary')
        for i, n in activities.items()
        for j in range(B)
    }
    x = {
        (i, k): pulp.LpVariable(f"x_{i}_{k}", cat='Binary')
        for i in activities
        for k in range(B)
    }

    # 3. Restricciones de inicio único y rango válido
    for i, n in activities.items():
        for j in range(B):
            if j > B - n:
                model += s[(i, j)] == 0
        model += pulp.lpSum(s[(i, j)] for j in range(B - n + 1)) == 1

    # 4. Relación continuidad: x[i,k] = suma de inicios que cubren k
    for i, n in activities.items():
        for k in range(B):
            model += x[(i, k)] == pulp.lpSum(
                s[(i, j)]
                for j in range(max(0, k - n + 1), min(k + 1, B - n + 1))
            )

    # 5. Demanda exacta de días por actividad
    for i, n in activities.items():
        model += pulp.lpSum(x[(i, k)] for k in range(B)) == n

    # 6. Carga diaria uniforme
    total_days = sum(activities.values())
    floor_avg = total_days // B
    ceil_avg = -(-total_days // B)
    for k in range(B):
        model += pulp.lpSum(x[(i, k)] for i in activities) >= floor_avg
        model += pulp.lpSum(x[(i, k)] for i in activities) <= ceil_avg

    # 7. Objetivo dummy (factibilidad)
    model += 0

    # 8. Resolver
    model.solve()

    # 9. Construir DataFrame de salida
    df = pd.DataFrame(
        index=activities.keys(),
        columns=[d.strftime("%Y-%m-%d") for d in business_days]
    )
    for i in activities:
        for k, d in enumerate(business_days):
            df.at[i, d.strftime("%Y-%m-%d")] = int(pulp.value(x[(i, k)]))

    return df

# Ejemplo de uso:
# activities = {'actividad1':5, 'actividad2':12, 'actividad4':3, 'actividad5':9, 'actividad6':11}
# holidays = [date(2025,5,1), date(2025,5,15)]
# df = distribuir_actividades_ilp(activities, 2025, 5, holidays)


In [132]:
df_pruebas= distribuir_actividades_ilp(dfs_dict_filtrado["2025-05"]["dias"].to_dict(), 2025, 5, holidays_2025)
df_pruebas.head(20)

,2025-05-01,2025-05-05,2025-05-06,2025-05-07,2025-05-08,2025-05-09,2025-05-12,2025-05-13,2025-05-14,2025-05-15,...,2025-05-19,2025-05-20,2025-05-21,2025-05-22,2025-05-23,2025-05-26,2025-05-27,2025-05-28,2025-05-29,2025-05-30
MR101,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MR102,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,1
MR104,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MR201,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
MR203,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
MR301,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
MR401,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
MR701,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:

def distribuir_control_vigilancia_ilp(activities, year, month, holidays):
    """
    Distribuye actividades de vigilancia (MR601) en los últimos viernes (o jueves si feriado) del mes
    usando programación lineal entera (pulp), maximizando proximidad al fin de mes.

    Parámetros:
    - activities: dict de actividad → número de días
    - year: int
    - month: int
    - holidays: list[date]

    Retorna:
    - DataFrame con índice actividades y columnas fechas (YYYY-MM-DD), valores 1 o 0.
    """
    # Generar todos los días del mes
    start_date = date(year, month, 1)
    end_date = (pd.Timestamp(start_date) + pd.offsets.MonthEnd(0)).date()
    all_days = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Buscar candidatos: viernes no feriados, o jueves anterior si viernes es feriado
    candidate_days = []
    weights = {}
    for d in all_days:
        d_date = d.date()
        if d_date.weekday() == 4:  # viernes
            if d_date in holidays:
                # buscar jueves anterior
                d2 = d_date - timedelta(days=1)
                while d2 in holidays or d2.weekday() > 4:
                    d2 -= timedelta(days=1)
                candidate_days.append(d2)
            else:
                candidate_days.append(d_date)

    # Eliminar duplicados y ordenar
    candidate_days = sorted(set(candidate_days))
    for d in candidate_days:
        # Peso mayor para días más cercanos al fin de mes
        weights[d] = (d - start_date).days

    # Crear DataFrame con ceros
    df = pd.DataFrame(0, index=activities.keys(), columns=[d.date().isoformat() for d in all_days])

    for actividad, n_dias in activities.items():
        # Crear modelo ILP
        model = pulp.LpProblem("Distribucion_Vigilancia", pulp.LpMaximize)
        x = {d: pulp.LpVariable(f"x_{d}", cat='Binary') for d in candidate_days}

        # Objetivo: maximizar suma de pesos
        model += pulp.lpSum([weights[d] * x[d] for d in candidate_days])

        # Restricción: solo n_dias deben ser seleccionados
        model += pulp.lpSum([x[d] for d in candidate_days]) == n_dias

        # Resolver
        model.solve()

        # Aplicar solución al DataFrame
        for d in candidate_days:
            if x[d].value() == 1:
                df.loc[actividad, d.isoformat()] = 1

    return df

# Ejecutar con 4 días en mayo 2025
distribuir_control_vigilancia_ilp({"MR601": 4}, 2025, 5, [date(2025, 5, 2)])


,2025-05-01,2025-05-02,2025-05-03,2025-05-04,2025-05-05,2025-05-06,2025-05-07,2025-05-08,2025-05-09,2025-05-10,...,2025-05-22,2025-05-23,2025-05-24,2025-05-25,2025-05-26,2025-05-27,2025-05-28,2025-05-29,2025-05-30,2025-05-31
MR601,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [134]:
for key, df_current in dfs_dict_filtrado.items():
    year, month = key.split("-")
    year = int(year)
    month = int(month)
    dias_en_mes = dias_mes_calculator(month, year)

    dias_dict = df_current["dias"].to_dict()
    # print(dias_dict)

    # distribucion de actividades normales
    df_distribucion = distribuir_actividades_ilp(dias_dict, year, month, holidays_2025)
    df_distribucion.head()

    # distribucion de actividades de vigilancia

    dias_vigilancia = monthly_activities_summary_df.loc["MR601", f"{year}-{month:02}"]
    df_distribucion_vigilancia = distribuir_control_vigilancia_ilp(
        {"MR601": dias_vigilancia}, year, month, holidays_2025
    )

    ## escribiendo cada archivo excel

    ws_month = wb.add_worksheet(str(month))

    h1 = concatenar_elementos(
        ["Cronograma de ejecución de actividades mes", nombre_mes(month), year]
    )

    header_label = concatenar_elementos(
        [
            "Formato N° 3",
        ]
    )

    header_contratista = concatenar_elementos(["Contratista:", datos["contratista"]])

    header_codigo_ruta = concatenar_elementos(["Código de ruta:", datos["ruta"]])

    header_codigo_tramo = concatenar_elementos(["Código de tramo:", datos["tramo"]])

    header_categoria = concatenar_elementos(["Categoría:", datos["categoria"]])

    header_jefe_mantenimiento = concatenar_elementos(
        ["Jefe de mantenimiento:", datos["jefe_mantenimiento"]]
    )

    header_longitud = concatenar_elementos(
        ["Longitud:", formatear_progresiva(datos["longitud"]), "Km"]
    )

    header_meta = concatenar_elementos(
        ["Meta:", formatear_progresiva(datos["meta"]), "Km"]
    )

    header_numero_trabajadores = concatenar_elementos(
        ["Número de trabajadores:", datos["numero_trabajadores"]]
    )

    header_sector = concatenar_elementos(["Sector:", datos["sector"]])

    header_cuadrilla = concatenar_elementos(["Cuadrilla:", datos["cuadrilla"]])

    # 3. Ajustar anchos de columna
    ws_month.set_column("A:A", 6)  # n
    ws_month.set_column("B:B", 10)  # Código
    ws_month.set_column("C:C", 35)  # Actividad
    ws_month.set_column("D:D", 8)  # Acti
    ws_month.set_column("E:E", 8)
    ws_month.set_column(f"F:{columna_final("F",dias_en_mes)}", 5)
    ws_month.set_column(
        f"{columna_final("F",dias_en_mes+1)}:{columna_final("F",dias_en_mes+1)}", 12
    )  # Total

    ws_month.merge_range(
        f"A4:{columna_final("F",dias_en_mes+1)}4", header_label, header_label_fmt
    )
    ws_month.merge_range(f"A5:{columna_final("F",dias_en_mes+1)}5", h1, header_fmt)

    ws_month.merge_range(f"A7:N7", header_contratista, header2_fmt)
    ws_month.merge_range(f"A8:N8", header_codigo_ruta, header2_fmt)
    ws_month.merge_range(f"A9:N9", header_codigo_tramo, header2_fmt)
    ws_month.merge_range(f"A10:N10", header_categoria, header2_fmt)
    ws_month.merge_range(f"A11:N11", header_jefe_mantenimiento, header2_fmt)

    ws_month.merge_range(f"P9:U9", header_longitud, header2_fmt)
    ws_month.merge_range(f"P10:U10", header_meta, header2_fmt)

    ws_month.merge_range(f"W7:AI7", header_numero_trabajadores, header2_fmt)
    ws_month.merge_range(f"W8:AI8", header_sector, header2_fmt)
    ws_month.merge_range(f"W9:AI9", header_cuadrilla, header2_fmt)

    # 5. Escribir encabezados de tabla

    ws_month.merge_range(f"A13:A16", "N.º", table_header_fmt)
    ws_month.merge_range(f"B13:B16", "Código", table_header_fmt)
    ws_month.merge_range(f"C13:C16", "Actividad", table_header_fmt)
    ws_month.merge_range(f"D13:D16", "Rend Unit.", table_header_fmt)
    ws_month.merge_range(f"E13:E16", "Unid.", table_header_fmt)

    ws_month.merge_range(
        f"F13:{columna_final("F",dias_en_mes+1)}13",
        f"Mes: {nombre_mes(month)}",
        table_header_fmt,
    )

    start_end_week = split_month_into_four_index_blocks(year, month)
    start_end_week1 = start_end_week[0]
    start_end_week2 = start_end_week[1]
    start_end_week3 = start_end_week[2]
    start_end_week4 = start_end_week[3]

    ws_month.merge_range(
        f"{columna_final("F",start_end_week1[0]+1)}14:{columna_final("F",start_end_week1[1]+1)}14",
        f"Semana 1",
        table_header_fmt,
    )

    ws_month.merge_range(
        f"{columna_final("F",start_end_week2[0]+1)}14:{columna_final("F",start_end_week2[1]+1)}14",
        f"Semana 2",
        table_header_fmt,
    )

    ws_month.merge_range(
        f"{columna_final("F",start_end_week3[0]+1)}14:{columna_final("F",start_end_week3[1]+1)}14",
        f"Semana 3",
        table_header_fmt,
    )

    ws_month.merge_range(
        f"{columna_final("F",start_end_week4[0]+1)}14:{columna_final("F",start_end_week4[1]+1)}14",
        f"Semana 4",
        table_header_fmt,
    )

    ws_month.merge_range(
        f"{columna_final("F",dias_en_mes+1)}14:{columna_final("F",dias_en_mes+1)}16",
        "Total",
        table_header_fmt,
    )

    # ——— Map de iniciales en español (lunes=0 … domingo=6) ———
    # cambiamos la de miércoles por “X”
    iniciales = ["L", "M", "X", "J", "V", "S", "D"]

    # ——— Número de días y día de la semana del día 1 ———
    # first_weekday: 0=Lunes, …, 6=Domingo; num_days: 28–31
    first_weekday, num_days = calendar.monthrange(year, month)

    # ——— Escribir encabezados: iniciales y números ———
    # Empezamos en la columna 0 (A). Si quieres otro desplazamiento, añade un 'offset_col'.
    offset_row = 14  # << cuántas filas bajas quieres dejar en blanco
    offset_col = 5  # << cuántas columnas a la derecha

    # filas donde van los encabezados y los números, usando el offset vertical
    fila_inicial = offset_row
    fila_dia = offset_row + 1

    for day in range(1, num_days + 1):
        wd = date(year, month, day).weekday()  # 0=Lunes … 6=Domingo
        col = offset_col + (day - 1)
        # escribimos la inicial en la fila de encabezados
        ws_month.write(fila_inicial, col, iniciales[wd], table_header_fmt)
        # escribimos el número del día justo debajo
        ws_month.write(fila_dia, col, day, table_header_number_integer_fmt)

  

    rows = []
    for sección in partidas:
        # fila de sección (solo código y descripción, resto vacío)
        rows.append(
            {
                "N": "",
                "Código": sección["codigo"],
                "Actividad": sección["descripcion"],
                "Rend Unit.": "",
                "Unid.": "",
            }
        )
        # filas de actividades
        for indice, act in enumerate(sección["actividades"]):
            rows.append(
                {
                    "N": "",
                    "Código": act["codigo"],
                    "Actividad": act["descripcion"],
                    "Rend Unit.": act["rendimiento"],
                    "Unid.": act["unidad_medida"],
                }
            )

    # 2. Punto de inicio en B11
    start_row, start_col = 16, 0  # B11 en 0‑based

    # 3. Recorre rows y escribe
    for index_row, row in enumerate(rows):
        r = start_row + index_row
        # si es fila de sección (unidad vacía), negrita; si no, formato normal
        is_seccion = row["Unid."] == ""
        fmt = section_fmt if is_seccion else no_section_fmt

        # columna B → Código
        ws_month.write(r, start_col + 0, index_row+1, fmt)
        ws_month.write(r, start_col + 1, row["Código"], fmt)
        # columna C → Actividad (con sangría si no es sección)
        ws_month.write(
            r,
            start_col + 2,
            row["Actividad"],
            section_fmt if is_seccion else activity_indent,
        )
        # resto de columnas D‑H
        ws_month.write(r, start_col + 3, row["Rend Unit."], fmt)
        ws_month.write(r, start_col + 4, row["Unid."], fmt)
        # ws.write(r, start_col + 5, row["Nº Cuad."], fmt)

    start_col = 5  # columna F (1-based)
    start_row = 16  # fila 16 (1-based)

    for index_row, row in enumerate(rows):
        if row["Código"] in df_distribucion.index:
            codigo = row["Código"]
            for i in range(0, dias_en_mes):
                dia = i + 1
                fila = codigo
                fecha = f"{year}-{month:02d}-{dia:02d}"

                if fecha in df_distribucion.columns:
                    valor = df_distribucion.loc[codigo, fecha]

                    if(valor == 1):
                        carga_diaria=dfs_por_mes[f"{year}-{month:02d}"].loc[codigo, "carga_por_dia"]
                        ws_month.write(start_row + index_row, start_col + i, carga_diaria, cell_fmt)

    # es ineficiente porque solo deberia analizar 
    # su fila
    start_col = 5  # columna F (1-based)
    start_row = 16  # fila 16 (1-based)
    for index_row, row in enumerate(rows):
        if row["Código"] in df_distribucion_vigilancia.index:
            codigo = row["Código"]
            for i in range(0, dias_en_mes):
                dia = i + 1
                fila = codigo
                fecha = f"{year}-{month:02d}-{dia:02d}"

                if fecha in df_distribucion_vigilancia.columns:
                    valor = df_distribucion_vigilancia.loc[codigo, fecha]

                    if(valor == 1):
                        carga_diaria=dfs_por_mes[f"{year}-{month:02d}"].loc[codigo, "carga_por_dia"]
                        ws_month.write(start_row + index_row, start_col + i, carga_diaria, cell_fmt)

    filas = [18, 19,20,21,23,34,25,26,27,28,30,32,34,36,38,39]
    for fila in filas:
        celda_destino = f"{columna_final("F",dias_en_mes+1)}{fila}"
        formula = f"=SUM(F{fila}:{columna_final("F",dias_en_mes)}{fila})"
        ws_month.write_formula(celda_destino, formula,suma_fmt)

    # AGREGANDO BORDE DE LA TABLA
    ws_month.conditional_format(F"F17:{columna_final("F",dias_en_mes+1)}39", {"type": "no_errors", "format": cell_fmt})



## Resumen

In [135]:
h1= concatenar_elementos([
    'Resumen de programación mensual',
    datos['fecha_inicio']['year']
])
header_label=concatenar_elementos([
    'Formato N° 03.02',
])

header_servicio=concatenar_elementos([
    'Servicio:',
    datos['servicio']
])

header_tramo=concatenar_elementos([
    'Tramo:',
    datos['tramo']
])

header_longitud=concatenar_elementos([
    'Longitud:',
    formatear_progresiva(datos['longitud'])
])

header_plazo_ejecucion=concatenar_elementos([
    'Plazo de ejecución:',
    datos['tiempo_ejecucion_dias'],
    'días calendarios'
])

header_numero_cuadrillas=concatenar_elementos([
    'Número de cuadrillas:',
    datos['numero_cuadrillas']
])

header_numero_trabajadores=concatenar_elementos([
    'Número de trabajadores:',
    datos['numero_trabajadores']
])

header_codigo_ruta=concatenar_elementos([
    'Código de ruta:',
    datos['ruta']
])






In [136]:
ws_summary = wb.add_worksheet("resumen")

# Ajustar anchos de columna
ws_summary.set_column("A:A", 8)  # N
ws_summary.set_column("B:B", 10)  # Código
ws_summary.set_column("C:C", 35)  # Actividad
ws_summary.set_column("D:D", 8)  # Unidad
ws_summary.set_column("E:E", 8)  # Rend. Unit.

month_start=fecha_inicio.month
month_end=fecha_fin.month

r = relativedelta( fecha_fin,fecha_inicio)

num_months = r.years * 12 + r.months + 1

ws_summary.set_column(f"F:{columna_final('F',num_months+1)}", 10)  # Meses

ws_summary.set_column(f"{columna_final('F',num_months+1)}:{columna_final('F',num_months+1)}", 22)  # Total

ws_summary.set_column(f"{columna_final('F',num_months+1)}:{columna_final('F',num_months+2)}", 22)  # Progresivas

# Escribir titulos

ws_summary.merge_range(f"A1:{columna_final('F',num_months+2)}1", h1, header_fmt)
ws_summary.merge_range(f"A2:{columna_final('F',num_months+2)}2", header_label, header_label_fmt)

ws_summary.merge_range("A4:J4", header_servicio, header2_fmt)
ws_summary.merge_range("A5:J5", header_tramo, header2_fmt)
ws_summary.merge_range("A6:J6", header_longitud, header2_fmt)
ws_summary.merge_range("A7:J7", header_plazo_ejecucion, header2_fmt)
ws_summary.merge_range("A8:J8", header_numero_cuadrillas, header2_fmt)
ws_summary.merge_range("A9:J9", header_numero_trabajadores, header2_fmt)
ws_summary.merge_range("A10:J10", header_codigo_ruta, header2_fmt)


# Escribir encabezados de tabla

ws_summary.merge_range(f"A12:A14", "N.º", table_header_fmt)
ws_summary.merge_range(f"B12:B14", "Código", table_header_fmt)
ws_summary.merge_range(f"C12:C14", "Actividad", table_header_fmt)
ws_summary.merge_range(f"D12:D14", "Unidad", table_header_fmt)
ws_summary.merge_range(f"E12:E14", "Rend. Unit./Cuadrilla", table_header_fmt)

ws_summary.merge_range(f"F12:{columna_final('F',num_months)}12", "Meses", table_header_fmt)

for i in range(0, num_months):
    mes = month_start + i
    if mes > 12:
        mes = mes - 12
    ws_summary.write(12, 5 + i, nombre_mes(mes), table_header_fmt)
    ws_summary.write(13, 5 + i, f"{i+1}.º mes", table_header_fmt)


ws_summary.merge_range(f"{columna_final('F',num_months+1)}12:{columna_final('F',num_months+1)}14", "Total de cargas de trabajo programado", table_header_fmt)

ws_summary.merge_range(f"{columna_final('F',num_months+2)}12:{columna_final('F',num_months+2)}14", "Progresivas", table_header_fmt)

rows=[]

for sección in partidas:
    # fila de sección (solo código y descripción, resto vacío)
    rows.append(
        {
            "N": "",
            "Código": sección["codigo"],
            "Actividad": sección["descripcion"],
            "Unidad": "",
            "Rend. Unit./Cuadrilla": "",
        }
    )
    # filas de actividades
    for act in sección["actividades"]:
        rows.append(
            {
                "N": "",
                "Código": act["codigo"],
                "Actividad": act["descripcion"],
                "Unidad": act["unidad_medida"],
                "Rend. Unit./Cuadrilla": act["rendimiento"]*datos['numero_trabajadores'],
            }
        )

# 2. Punto de inicio en B11
start_row, start_col = 14, 0  # B11 en 0‑based

# 3. Recorre rows y escribe

for index_row, row in enumerate(rows):
    r = start_row + index_row
    # si es fila de sección (unidad vacía), negrita; si no, formato normal
    is_seccion = row["Unidad"] == ""
    fmt = section_fmt if is_seccion else no_section_fmt

    # columna B → Código
    ws_summary.write(r, start_col + 0, index_row+1, fmt)
    ws_summary.write(r, start_col + 1, row["Código"], fmt)
    # columna C → Actividad (con sangría si no es sección)
    ws_summary.write(
        r,
        start_col + 2,
        row["Actividad"],
        section_fmt if is_seccion else activity_indent,
    )
    # resto de columnas D‑H
    ws_summary.write(r, start_col + 3, row["Unidad"], fmt)
    ws_summary.write(r, start_col + 4, row["Rend. Unit./Cuadrilla"], fmt)

initial_row = 15

offset_col = 17  # cuántas columnas a la derecha
offset_row = 5  # cuántas filas bajas quieres dejar en blanco

for i in range(0, num_months):
    mes = month_start + i
    for j in range(0, len(rows)):
        escribir_en = f"{columna_final('F', i+1)}{initial_row+j}"
        days_en_mes = dias_mes_calculator(mes, year)
        copiar_de = f"{columna_final('F', days_en_mes+1)}{offset_col+j}"

        ws_summary.write_formula(escribir_en, f"='{str(mes)}'!{copiar_de}", cell_number_fmt)

filas = [16,17,18,19,21,22,23,24,25,26,28,30,32,34,36,37]

for fila in filas:
    celda_destino = f"{columna_final('F',num_months+1)}{fila}"
    formula = f"=SUM(F{fila}:{columna_final('F',num_months)}{fila})"
    ws_summary.write_formula(celda_destino, formula,suma_fmt)

for i in range(0, len(rows)):
    ws_summary.write(f"{columna_final('F', num_months+2)}{initial_row+i}", f"00+000 - {formatear_progresiva(datos['longitud'])}", cell_align_right_fmt)

## Cronograma de desombolso

In [138]:

pagos=calcular_pagos_por_mes(fecha_inicio,datos['tiempo_ejecucion_dias'],datos['monto'])

print(pagos)


{4: 5678.4, 5: 8382.4, 6: 8112.0, 7: 8382.4, 8: 8382.4, 9: 8112.0, 10: 8382.4, 11: 8112.0, 12: 1352.0}


In [139]:
ws_desembolso = wb.add_worksheet("desembolso")

num_months = contar_meses_obra(fecha_inicio, datos["tiempo_ejecucion_dias"])

h1 = concatenar_elementos(["Cronograma de desembolso", datos["fecha_inicio"]["year"]])

# Ajustar anchos de columna
ws_desembolso.set_column("A:A", 8)  # ST
ws_desembolso.set_column("B:B", 15)  # Departamento
ws_desembolso.set_column("C:C", 15)  # Provincia
ws_desembolso.set_column("D:D", 15)  # Distrito
ws_desembolso.set_column("E:E", 25)  # Camnino Vecinal

ws_desembolso.set_column("F:F", 15)  # Contrato
ws_desembolso.set_column("G:G", 15)  # Longitud
ws_desembolso.set_column("H:H", 15)  # Plazo de ejecución
ws_desembolso.set_column("I:I", 15)  # Fecha de inicio
ws_desembolso.set_column("J:J", 15)  # Monto contratado

ws_desembolso.set_column("K:K", 15)  # Adenda
ws_desembolso.set_column("L:L", 15)  # Monto adenda

ws_desembolso.set_column(
    f"M:{columna_final('M',num_months*2+2)}", 20
)  # Programación de gasto

# Ajustar alto de fila
ws_desembolso.set_row(0, 30)  # Alto de la fila 0 (encabezado)
# ws_desembolso.set_row(1, 30)  # Alto de la fila 1 (subencabezado)
ws_desembolso.set_row(2, 40)  # Alto de la fila 2 (subencabezado)
ws_desembolso.set_row(3, 40)  # Alto de la fila 3 (subencabezado)
ws_desembolso.set_row(4, 40)  # Alto de la fila 4 (subencabezado)
ws_desembolso.set_row(5, 60)  # Alto de la fila 5 (subencabezado)

# Escribir titulos
ws_desembolso.merge_range(f"A1:{columna_final("M",num_months*2)}1", h1, header_fmt)

# Escribir encabezados de tabla

ws_desembolso.merge_range(f"A3:E4", "Datos generales", table_header_fmt)

ws_desembolso.write("A5", "ST", table_header_fmt)
ws_desembolso.write("B5", "Departamento", table_header_fmt)
ws_desembolso.write("C5", "Provincia", table_header_fmt)
ws_desembolso.write("D5", "Distrito", table_header_fmt)
ws_desembolso.write("E5", "Camino Vecinal", table_header_fmt)

ws_desembolso.merge_range(f"F3:J4", "Contrato", table_header_fmt)

ws_desembolso.write("F5", "Contrato", table_header_fmt)
ws_desembolso.write("G5", "Longitud", table_header_fmt)
ws_desembolso.write("H5", "Plazo de ejecución", table_header_fmt)
ws_desembolso.write("I5", "Fecha de inicio", table_header_fmt)
ws_desembolso.write("J5", "Monto contratado", table_header_fmt)


ws_desembolso.merge_range(f"K3:L4", "Adenda", table_header_fmt)
ws_desembolso.write("K5", "Adenda", table_header_fmt)
ws_desembolso.write("L5", "Monto adenda", table_header_fmt)

# Contenido de la tabla
ws_desembolso.write("A6", "1", cell_text_break_fmt)
ws_desembolso.write("B6", datos["departamento"], cell_text_break_fmt)
ws_desembolso.write("C6", datos["provincia"], cell_text_break_fmt)
ws_desembolso.write("D6", " ".join(datos["distritos"]), cell_text_break_fmt)
ws_desembolso.write("E6", datos["ruta"], cell_text_break_fmt)

ws_desembolso.write("F6", datos["contrato"], cell_text_break_fmt)
ws_desembolso.write("G6", formatear_progresiva(datos["longitud"]), cell_text_break_fmt)
ws_desembolso.write("H6", datos["tiempo_ejecucion_dias"], cell_text_break_fmt)
ws_desembolso.write("I6", fecha_inicio.strftime("%d/%m/%Y"), cell_text_break_fmt)
ws_desembolso.write("J6", f"S/. {datos['monto']}", cell_text_break_fmt)
ws_desembolso.write("K6", "-", cell_text_break_fmt)
ws_desembolso.write("L6", "-", cell_text_break_fmt)


# Llenado de valores por mes
row_start = "M"

ws_desembolso.merge_range(
    f"{row_start}3:{columna_final("M",num_months*2+2)}3",
    f"Programación de gasto {datos['fecha_inicio']['year']}",
    table_header_fmt,
)

for i in range(0, num_months):
    mes = month_start + i
    if mes > 12:
        mes = mes - 12

    ws_desembolso.merge_range(
        f"{columna_final(row_start,i*2+1)}4:{columna_final(row_start,i*2+2)}4",
        f"{nombre_mes(mes)}",
        table_header_fmt,
    )

    ws_desembolso.write(f"{columna_final(row_start,i*2+1)}5", "Mant.", table_header_fmt)
    ws_desembolso.write(f"{columna_final(row_start,i*2+2)}5", "G.O.", table_header_fmt)
    ws_desembolso.write(f"{columna_final(row_start,i*2+1)}6", pagos[mes], soles_fmt)
    ws_desembolso.write(
        f"{columna_final(row_start,i*2+2)}6", pagos[mes] * 0.1, soles_fmt
    )
    ws_desembolso.write(f"{columna_final(row_start,i*2+1)}7", pagos[mes], soles_fmt)
    ws_desembolso.write(
        f"{columna_final(row_start,i*2+2)}7", pagos[mes] * 0.1, soles_fmt
    )

    ws_desembolso.merge_range(
        f"{columna_final(row_start,i*2+1)}8:{columna_final(row_start,i*2+2)}8",
        pagos[mes] * 1.1,
        soles_color_fmt,
    )

# suma de los meses


ws_desembolso.merge_range(
    f"{columna_final(row_start,num_months*2+1)}4:{columna_final(row_start,num_months*2+2)}4",
    "Mant.",
    table_header_fmt,
)
ws_desembolso.merge_range(
    f"{columna_final(row_start,num_months*2+1)}5:{columna_final(row_start,num_months*2+2)}5",
    "G.O.",
    table_header_fmt,
)

formula_suma_mant=generar_suma_intercalada("M6", num_months)
formula_suma_go=generar_suma_intercalada("N6", num_months )

formula_suma_mant_total=generar_suma_intercalada("M7", num_months)
formula_suma_go_total=generar_suma_intercalada("N7", num_months)

formula_suma_total=generar_suma_intercalada("M8", num_months)

ws_desembolso.write_formula(f"{columna_final(row_start,num_months*2+1)}6", formula_suma_mant, soles_color_fmt)
ws_desembolso.write(f"{columna_final(row_start,num_months*2+2)}6", formula_suma_go, soles_color_fmt)

ws_desembolso.write_formula(f"{columna_final(row_start,num_months*2+1)}7", formula_suma_mant_total, soles_color_fmt)
ws_desembolso.write(f"{columna_final(row_start,num_months*2+2)}7", formula_suma_go_total, soles_color_fmt)

ws_desembolso.merge_range(f"{columna_final(row_start,num_months*2+1)}8:{
    columna_final(row_start,num_months*2+2)}8", formula_suma_total, soles_color_fmt)


0

In [140]:
wb.close()
